In [ ]:
import numpy as np
import plotly.graph_objects as go

# Load the data
dat = np.load('/home/maria/LuckyMouse2/pixel_transformer_neuro/data/processed/hybrid_neural_responses_reduced.npy')  # shape: (neurons, 118)

# Convert to probabilities
prob_matrix = dat / 50.0  # now values in [0,1]
epsilon = 1e-10
prob_matrix_safe = prob_matrix + epsilon
num_neurons, num_images = prob_matrix.shape

# KL divergence from uniform
uniform_dist = np.full((num_images,), 1.0 / num_images)
kl_divs = []
for neuron_probs in prob_matrix_safe:
    neuron_dist = neuron_probs / np.sum(neuron_probs)
    kl = np.sum(uniform_dist * np.log(uniform_dist / neuron_dist))
    kl_divs.append(kl)
kl_divs = np.array(kl_divs)

# Bin KL values
bin_edges = np.histogram_bin_edges(kl_divs, bins=30)
bin_indices = np.digitize(kl_divs, bins=bin_edges) - 1  # get bin for each neuron

# Collect hover text: first neuron in each bin
hover_texts = [''] * (len(bin_edges) - 1)
bin_counts = np.zeros(len(hover_texts), dtype=int)

for idx, bin_idx in enumerate(bin_indices):
    if 0 <= bin_idx < len(hover_texts):
        bin_counts[bin_idx] += 1
        if hover_texts[bin_idx] == '':
            sorted_events = np.sort(dat[idx])
            hover_texts[bin_idx] = f"Bin range: {bin_edges[bin_idx]:.2f}–{bin_edges[bin_idx+1]:.2f}<br>" \
                                   f"Sorted events: {sorted_events.tolist()}"

# Replace empty bins with placeholders
for i in range(len(hover_texts)):
    if hover_texts[i] == '':
        hover_texts[i] = f"Bin range: {bin_edges[i]:.2f}–{bin_edges[i+1]:.2f}<br>No neurons in bin."

# Plotly histogram (manual binning to attach hover text)
fig = go.Figure(data=[
    go.Bar(
        x=[f"{bin_edges[i]:.2f}–{bin_edges[i+1]:.2f}" for i in range(len(bin_edges)-1)],
        y=bin_counts,
        text=hover_texts,
        hoverinfo='text'
    )
])

fig.update_layout(
    title="KL Divergence from Uniform Distribution (per Neuron)",
    xaxis_title="KL Divergence Bin",
    yaxis_title="Number of Neurons",
    xaxis_tickangle=45,
    bargap=0.1,
)

fig.show()


In [ ]:
import numpy as np
import plotly.graph_objects as go
import ipywidgets as widgets
from IPython.display import display, clear_output

# Load your neural response data
dat = np.load('/home/maria/LuckyMouse2/pixel_transformer_neuro/data/processed/hybrid_neural_responses_reduced.npy')  # shape: (neurons, 118)
prob_matrix = dat / 50.0
epsilon = 1e-10
prob_matrix_safe = prob_matrix + epsilon
num_neurons, num_images = prob_matrix.shape

# Compute KL divergence from uniform
uniform_dist = np.full((num_images,), 1.0 / num_images)
kl_divs = []
for neuron_probs in prob_matrix_safe:
    neuron_dist = neuron_probs / np.sum(neuron_probs)
    kl = np.sum(uniform_dist * np.log(uniform_dist / neuron_dist))
    kl_divs.append(kl)
kl_divs = np.array(kl_divs)

# Bin setup
num_bins = 30
bin_edges = np.histogram_bin_edges(kl_divs, bins=num_bins)
bin_indices = np.digitize(kl_divs, bins=bin_edges) - 1

# Prepare data for each bin
bin_to_neuron_indices = {i: [] for i in range(num_bins)}
for idx, bin_idx in enumerate(bin_indices):
    if 0 <= bin_idx < num_bins:
        bin_to_neuron_indices[bin_idx].append(idx)

bin_counts = [len(bin_to_neuron_indices[i]) for i in range(num_bins)]
bin_labels = [f"{bin_edges[i]:.2f}–{bin_edges[i+1]:.2f}" for i in range(num_bins)]

# Widget for outputting the secondary plot
output_plot = widgets.Output()

# Define the click handler
def on_bar_click(trace, points, state):
    with output_plot:
        clear_output(wait=True)
        if points.point_inds:
            bin_idx = points.point_inds[0]
            neuron_ids = bin_to_neuron_indices.get(bin_idx, [])
            if neuron_ids:
                neuron_id = neuron_ids[0]
                sorted_events = np.sort(dat[neuron_id])
                fig_detail = go.Figure()
                fig_detail.add_trace(go.Bar(
                    x=list(range(len(sorted_events))),
                    y=sorted_events,
                    marker_color='mediumpurple'
                ))
                fig_detail.update_layout(
                    title=f"Sorted Event Counts for First Neuron in Bin {bin_labels[bin_idx]}",
                    xaxis_title="Sorted Stimulus Index",
                    yaxis_title="Event Count",
                    height=300
                )
                fig_detail.show()
            else:
                print("No neurons in this bin.")

# Create main histogram figure
fig = go.FigureWidget(
    data=[go.Bar(x=bin_labels, y=bin_counts, marker_color='lightskyblue')],
    layout=dict(
        title="KL Divergence from Uniform Distribution (per Neuron)",
        xaxis_title="KL Divergence Bin",
        yaxis_title="Number of Neurons",
        xaxis_tickangle=45,
        bargap=0.1
    )
)

# Attach the click callback
fig.data[0].on_click(on_bar_click)

# Display in notebook
display(fig)
display(output_plot)


ImportError: Please install anywidget to use the FigureWidget class